In [54]:
import pandas as pd
import numpy as np
house_df = pd.read_csv('./data/Ames_Housing_Price_Data.csv',index_col = 0)
real_estate_df = pd.read_csv('./data/Ames_Real_Estate_Data.csv', index_col = 0, low_memory= False)

Notes:
Tons of multicollinearity. 

Might be a good idea to merge like columns (i.e. basement, garage, pool, fence, Kitchen, fireplace) into one. 

There are some columns which are basically exactly the same. So cut those.


In [18]:
house_df[house_df.columns]

Index(['PID', 'GrLivArea', 'SalePrice', 'MSSubClass', 'MSZoning',
       'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond

In [17]:
house_df.isna().sum()[house_df.isna().sum() > 0]

LotFrontage      462
Alley           2412
MasVnrType        14
MasVnrArea        14
BsmtQual          69
BsmtCond          69
BsmtExposure      71
BsmtFinType1      69
BsmtFinSF1         1
BsmtFinType2      70
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
FireplaceQu     1241
GarageType       127
GarageYrBlt      129
GarageFinish     129
GarageCars         1
GarageArea         1
GarageQual       129
GarageCond       129
PoolQC          2571
Fence           2055
MiscFeature     2483
dtype: int64

In [37]:
house_df[['Exterior1st','Exterior2nd']][house_df.Exterior1st != house_df.Exterior2nd] 
#not a lot of differences here... maybe cut one of the columns? Basically the same? 

,Exterior1st,Exterior2nd
8,WdShing,Wd Shng
9,HdBoard,Plywood
23,CemntBd,CmentBd
35,BrkFace,Wd Sdng
45,Wd Sdng,HdBoard
...,...,...
714,HdBoard,Plywood
729,WdShing,Wd Shng
741,WdShing,Wd Shng
758,HdBoard,Stucco


In [45]:
house_df[['BsmtFinSF1','BsmtFinSF2']][house_df.BsmtFinSF1 != house_df.BsmtFinSF2] #less commonly the same. So 

,BsmtFinSF1,BsmtFinSF2
1,238.0,0.0
2,552.0,393.0
3,737.0,0.0
5,643.0,0.0
9,647.0,0.0
...,...,...
755,890.0,0.0
757,574.0,0.0
761,284.0,0.0
762,841.0,0.0


In [46]:
house_df[['Condition1','Condition2']][house_df.Condition1 != house_df.Condition2] 
#Mostly the same. Think about dropping condition 2

,Condition1,Condition2
26,RRAn,Norm
29,PosN,Norm
31,Artery,Norm
44,RRAe,Norm
49,Artery,Norm
...,...,...
735,Artery,Norm
746,RRAn,Norm
752,Artery,Norm
755,RRAe,Norm


In [42]:
house_df.Neighborhood.unique() #Neighborhoods

array(['SWISU', 'Edwards', 'IDOTRR', 'OldTown', 'NWAmes', 'Blmngtn',
       'Mitchel', 'NridgHt', 'NAmes', 'CollgCr', 'SawyerW', 'MeadowV',
       'BrDale', 'Gilbert', 'Timber', 'Somerst', 'BrkSide', 'NoRidge',
       'Sawyer', 'Crawfor', 'Greens', 'ClearCr', 'GrnHill', 'StoneBr',
       'Veenker', 'NPkVill', 'Blueste', 'Landmrk'], dtype=object)

In [47]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [142]:
dumb_df = pd.get_dummies(house_df.drop('LotFrontage',axis = 1))
dumb_matrix = np.concatenate((np.ones((house_df.shape[0],1)), dumb_df.values), axis = 1)
dumb_matrix[np.isnan(dumb_matrix)] = 0

vif_vals = []
for j in range(dumb_matrix.shape[1]):
    vif_vals.append(variance_inflation_factor(dumb_matrix, j))



c:\users\michael\appdata\local\programs\python\python38-32\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [176]:
[dumb_df.columns[i-1] for i in range(len(vif_vals)) if vif_vals[i] > 500]
#Not surprising. Just a ton of multicollinearity

['GrLivArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GarageYrBlt',
 'MSZoning_A (agr)',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_I (all)',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Grvl',
 'Street_Pave',
 'LotShape_IR1',
 'LotShape_IR2',
 'LotShape_IR3',
 'LotShape_Reg',
 'LandContour_Bnk',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_Lvl',
 'Utilities_AllPub',
 'Utilities_NoSewr',
 'LotConfig_Corner',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'LandSlope_Gtl',
 'LandSlope_Mod',
 'LandSlope_Sev',
 'Neighborhood_Blmngtn',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_Gilbert',
 'Neighborhood_Greens',
 'Neighborhood_GrnHill',
 'Neighborhood_IDOTRR',
 'Neighborhood_Landmrk',
 'Neighborhood_MeadowV',
 'Neighborhood_Mi

In [157]:
dumb_df = pd.get_dummies(house_df.drop('LotFrontage',axis = 1))

In [174]:
dumb_df.columns[296]

'SaleCondition_Partial'

In [129]:
house_df.loc[:,'YearBuilt'] #majority of nan values
house_df.iloc[:,5]

1       NaN
2      42.0
3      60.0
4      80.0
5      70.0
       ... 
759     NaN
760     NaN
761    82.0
762     NaN
763    80.0
Name: LotFrontage, Length: 2580, dtype: float64

In [141]:
range(dumb_matrix.shape[1])

range(0, 298)

In [117]:
np.where(np.isnan(dumb_matrix))

(array([   0,   13,   17,   18,   22,   27,   28,   38,   40,   43,   44,
          48,   50,   54,   59,   60,   62,   66,   83,   86,   93,   94,
         104,  105,  106,  107,  109,  117,  120,  123,  131,  160,  163,
         165,  166,  175,  182,  187,  189,  195,  198,  209,  209,  219,
         227,  228,  229,  231,  236,  239,  249,  256,  275,  279,  285,
         288,  294,  305,  306,  308,  322,  324,  327,  336,  337,  339,
         346,  358,  359,  360,  362,  364,  369,  370,  372,  379,  389,
         393,  394,  395,  397,  399,  411,  412,  413,  424,  429,  431,
         433,  433,  433,  445,  447,  461,  465,  466,  468,  469,  470,
         471,  472,  482,  498,  509,  515,  516,  524,  528,  528,  529,
         530,  531,  533,  537,  540,  541,  542,  544,  549,  551,  556,
         557,  558,  565,  566,  568,  572,  576,  580,  587,  599,  604,
         613,  621,  624,  627,  635,  636,  642,  645,  650,  652,  653,
         654,  656,  662,  664,  668, 

In [135]:
house_df.drop('LotFrontage', axis = 1)

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotArea,Street,Alley,LotShape,LandContour,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,909176150,856,126000,30,RL,7890,Pave,NaN,Reg,Lvl,...,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
2,905476230,1049,139500,120,RL,4235,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
3,911128020,1001,124900,30,C (all),6060,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
4,535377150,1039,114000,70,RL,8146,Pave,NaN,Reg,Lvl,...,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
5,534177230,1665,227000,60,RL,8400,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,903205040,952,121000,30,RL,8854,Pave,NaN,Reg,Lvl,...,40,0,NaN,NaN,NaN,0,5,2009,WD,Normal
760,905402060,1733,139600,20,RL,13680,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
761,909275030,2002,145000,90,RH,6270,Pave,NaN,Reg,HLS,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
762,907192040,1842,217500,60,RL,8826,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal
